In [16]:
import numpy as np
import cv2

import time
from __future__ import print_function

import torch
from torch.autograd import Variable

from os.path import join
from glob import glob

import skimage.io as io
from skimage.transform import resize

from C3D_model import C3D

In [17]:
cuda = False
frames = 16

In [18]:
def get_blocc(vid):
    arr = []
    for i in range(frames):
        arr.append(vid.read()[1])

    blocc = np.array([resize(frame, output_shape=(112, 200), preserve_range=True) for frame in arr])

    blocc = blocc[:, :, 44:44+112, :]
    blocc = blocc.transpose(3, 0, 1, 2)  # ch, fr, h, w
    blocc = np.expand_dims(blocc, axis=0)  # batch axis
    blocc = np.float32(blocc)
    blocc = torch.from_numpy(blocc)
    
    return blocc

In [19]:
def read_labels_from_file(filepath):
    """
    Reads Sport1M labels from file
    
    Parameters
    ----------
    filepath: str
        the file.
        
    Returns
    -------
    list
        list of sport names.
    """
    with open(filepath, 'r') as f:
        labels = [line.strip() for line in f.readlines()]
    return labels

In [22]:
def predict(filename):
    # load a clip to be predicted
    # get network pretrained model
    net = C3D()
    net.load_state_dict(torch.load('c3d.pickle'))

    if cuda:
        net.cuda()
    net.eval()

    # perform prediction
    
    vid = cv2.VideoCapture(filename)
    ind = 0
    while(vid):
        X = get_blocc(vid)
        X = Variable(X)
        if cuda:
                X = X.cuda()
        start_time = time.time()
        prediction = net(X)
        prediction = prediction.data.cpu().numpy()
        #print("Time taken for prediction =", time.time()-start_time)
        ind = ind+1
        print(ind, end='\r', sep='\r')

        # read labels
        labels = read_labels_from_file('labels.txt')

        # print top predictions
        top_inds = prediction[0].argsort()[::-1][:5]  # reverse sort and take five largest items
        #print('\nTop 5:')
        #for i in top_inds:
        #  print('{:.5f} {}'.format(prediction[0][i], labels[i]))
        if(ind > 10):
            print(ind)
            break
    return (ind-1)*frames

In [23]:
start_time = time.time()
total_frames = predict('./vid_data/cycling.mp4')
total_time = time.time()-start_time
print("Total time taken = ", total_time)
print(total_frames/total_time, " Frames per second")

/home/nevin/.conda/envs/pytorch/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/nevin/.conda/envs/pytorch/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/nevin/nevin/projects/ml-exercises/paper-implemenations/c3d-pytorch/C3D_model.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = self.softmax(logits)


11
Total time taken =  9.346944332122803
17.117893753804147  Frames per second
